# Load dataset
Import Porto dataset from UP google drive.

(You should make a private copy of the dataset before running this script)

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls '/content/gdrive/My Drive'

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
porto-dataset  porto-dataset-2


# Retrain the model
## Import the file
Import the code from tensorflow retrain example

https://www.tensorflow.org/hub/tutorials/image_retraining

In [0]:
!curl -LO https://github.com/tensorflow/hub/raw/master/examples/image_retraining/retrain.py

## Display in Tensorboard

In [0]:
!ls 'gdrive/My Drive/porto-dataset'
!git clone https://github.com/mixuala/colab_utils

import os
import colab_utils.tboard

ROOT = %pwd
LOG_DIR = '/tmp/retrain_logs'

colab_utils.tboard.launch_tensorboard(bin_dir=ROOT, log_dir=LOG_DIR)

## Retrain on our dataset

In [0]:
!python retrain.py --image_dir 'gdrive/My Drive/porto-dataset/images'

# Display Results

## Analyse a single file

In [0]:
!curl -LO https://github.com/tensorflow/tensorflow/raw/master/tensorflow/examples/label_image/label_image.py
!python label_image.py \
  --graph=/tmp/output_graph.pb \
  --labels=/tmp/output_labels.txt \
  --input_layer=Placeholder \
  --output_layer=final_result \
  --image='gdrive/My Drive/porto-dataset/images/arrabida/arrabida-0010.jpg'

## Generate results per class

In [0]:
!curl -LO https://github.com/tensorflow/tensorflow/raw/master/tensorflow/examples/label_image/label_image.py
  
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse

import numpy as np
import tensorflow as tf
import label_image
import os

Set the needed variables.

In [0]:
model_file = '/tmp/output_graph.pb'
label_file = '/tmp/output_labels.txt'
input_layer = 'Placeholder'
output_layer = 'final_result'
images_path = 'gdrive/My Drive/porto-dataset/images'

Define the function that will calculate the percentage of correct values.

In [0]:
graph = label_image.load_graph(model_file)

def class_result_percentage(class_name):
  j = 0
  model_results = []
  class_path = os.path.join(images_path, class_name)
  for filename in os.listdir(class_path):
    file_name = os.path.join(class_path, filename)
    t = label_image.read_tensor_from_image_file(
        file_name,
        input_height=299,
        input_width=299,
        input_mean=0,
        input_std=255)

    input_name = "import/" + input_layer
    output_name = "import/" + output_layer
    input_operation = graph.get_operation_by_name(input_name)
    output_operation = graph.get_operation_by_name(output_name)

    with tf.Session(graph=graph) as sess:
      results = sess.run(output_operation.outputs[0], {
          input_operation.outputs[0]: t
      })
    results = np.squeeze(results)

    top_k = results.argsort()[-5:][::-1]
    labels = label_image.load_labels(label_file)
    image_results = []
    j += 1
    for i in top_k:
      image_results.append([labels[i], results[i]])
    model_results.append([j, image_results])

  correct_results = 0
  for image_results in model_results:
    if image_results[1][0][0] == class_name:
      correct_results += 1
  return 100 * correct_results / len(model_results)

In [0]:
classes = ['arrabida', 'camara', 'clerigos', 'musica', 'serralves']
    
for cn in classes:
  print(cn, class_result_percentage(cn))